In [1]:
import os
import sys

os.chdir("..")
sys.path.append("..")

In [2]:
import logging
import random
import yaml

import numpy as np
import torch

from src.models.utils import get_model
from src.models.trainer import Trainer

/home/hakon/miniforge3/envs/exprts/lib/python3.9/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(


In [3]:
def fit_model(config_path):
    with open(config_path, "r") as f:
        config = yaml.load(f, Loader=yaml.FullLoader)

    os.makedirs(config["path"], exist_ok=True)
    logging.basicConfig(
        filename=os.path.join(config["path"], "train.log"),
        level=logging.INFO,
        format="%(asctime)s %(message)s",
        datefmt="%m/%d/%Y %I:%M:%S",
    )

    random.seed(config["seed"])
    np.random.seed(config["seed"])
    torch.manual_seed(config["seed"])

    model_args = config["model_args"]
    trainer_args = config["trainer_args"]
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    model = get_model(config["model_name"])(
        **model_args, device=device, path=config["path"]
    ).to(device)
    trainer = Trainer(
        **trainer_args, sp=config["sp"], data_source=config["data_source"]
    )

    datadir = os.path.join("./data", config["dataset"])

    model.fit(trainer, datadir)


config_paths = [
    "experiments/electricity_nips/feedforward/config.yaml",
    "experiments/electricity_nips/feedforward_alternative/config.yaml",
    "experiments/electricity_nips/feedforward_augmented/config.yaml",
]

for config in config_paths:
    fit_model(config)